# Подключение

In [ ]:
import pyodbc

In [ ]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()

## Pandas

In [ ]:
import pandas as pd
from math import isnan

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [ ]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 150

## Необходимые функции

Позволяет получить список характеристик для всех предложений, указанных в list_of_keys. Предложения указываются в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

Преобразование из байтов в такую строку делается следующим образом:
```python
# type(x) == bytes
result = '0x' + x.hex().upper()
```

In [ ]:
def get_harks_by_object_keys(list_of_keys):
    """
    :param list_of_keys: список hex в виде '0x869CEA7372E...'
    :type list_of_keys: list
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из list_of_keys
    """
    
    in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
          t_har.Наименование,
          CASE 
              WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
              WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
              WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
              WHEN t.Значение_Тип = 01 THEN 
              CASE
                  WHEN t.Значение = 0x869CEA7372E5CA3E468F18025D610C0B THEN 'Да'
                  WHEN t.Значение = 0xB0CD188371B9A88E431B073454A9489F THEN 'Нет'
                  ELSE 'НетДанных'
              END
              WHEN t.Значение_Тип = 08 THEN 
              COALESCE(
                  (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
                  (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
              )
          END
              AS Значение,
          t.Объект
    FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
        JOIN [rway].[ПВХ].[Характеристики] t_har
            ON t_har.[Ссылка] = t.[Характеристика]
    WHERE Объект in {}
    '''.format(in_expr)
    
    cursor.execute(query)
    
    # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
    data = list(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), cursor.fetchall()))
    # Создание словаря для DataFrame
    d = [dict(map(lambda x: x[:2], filter(lambda y: y[-1] == offer_id, data)), **{'Ссылка': offer_id}) for offer_id in list_of_keys]
    df = pd.DataFrame(d)
    return df

    

Функция позволяет получить все предложения по конкретной задаче. Задача указывается в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

In [ ]:
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: Является полем "Ссылка" из таблицы, преобразованным из байтов в hex
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {}
          Код,
          Ссылка,
          АдресAhunter,
          АктуальнаяСсылкаИсточника,
          ДатаПересмотраЭкспозиции,
          ДатаПроверкиАктуальности,
          ДатаРазмещения,
          Город,
          Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] = (SELECT _IDRRef FROM [rway].[dbo].[_Task62] WHERE _FLD198 = '{}')

    '''.format(top, task_id)
    
    cursor.execute(query)
    data = cursor.fetchall()
    # Те же самые преобрзования, как в функции с характеристиками
    df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
    df.columns = list(map(lambda x: x[0], cursor.description))
    return df
    

# Боевой запуск

In [ ]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id('0001-0402-0001', offers_count=10000)

In [ ]:
base_df

In [ ]:
# Сбор характеристик по ссылкам из основного DataFrame base_df
har_df = get_harks_by_object_keys(list(base_df.loc[:, 'Ссылка']))

In [ ]:
# Собираем обе таблицы в одну большую
result_df = pd.merge(base_df, har_df, on='Ссылка', how='outer')

In [ ]:
result_df

Функция, в которой и происходит тестирование.

In [ ]:
def make_test():
    """
    :return: DataFrame, сшитый из основных характеристик и всех остальных
    """
    test_df = pd.DataFrame(index=result_df.columns)
    test_df['Рейтинг заполненности'] = [0] * len(test_df)
    test_df['Заполнено'] = [0] * len(test_df)
    test_df['Всего'] = [len(result_df)] * len(test_df)
    
    float_null = '0.000000'
    nan = str(float('nan'))
    
    for row in result_df:
        for cell in result_df.loc[:, row]:
            if cell and not cell == float_null and cell != nan and cell != 'НетДанных':  # Проверка на неподходящие значения
                test_df.loc[row, 'Заполнено'] += 1
                
    for row in test_df.iterrows():
        test_df.loc[row[0], 'Рейтинг заполненности'] = str(row[1]['Заполнено'] * 100 // row[1]['Всего']) + '%'
        
    return test_df

In [ ]:
test = make_test()

Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [ ]:
test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test

## Для тестов


In [ ]:
query = '''
SELECT
      _IDRRef
FROM [rway].[dbo].[_Task62] 
WHERE _FLD198 = '0001-0402-0001'


'''

cursor.execute(query)
data = cursor.fetchall()
df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
df.columns = list(map(lambda x: x[0], cursor.description))
df